# ElasticSearch & Python

Elasticsearch es un producto que permite indexar y analizar en tiempo real grandes cantidades de datos de manera distribuida. Va un paso más allá de la búsqueda por texto gracias a un DSL y un Api para búsquedas más complicadas.

A diferencia de otros motores de búsqueda y recuperación de información, ElasticSearch actúa como repositorio de información, almacenando los documentos que indexa. Esto permite reemplazar almacenes de documentos como MongoDB o Raven DB por ElasticSearch en nuestros proyectos, si lo consideramos necesario.
Tanto MongoDB como ElasticSearch son repositorios de documentos desnormalizados en los que los documentos que gestionan no necesitan disponer de un esquema rígido para poderlos introducir en el sistema.

Elasticsearch a diferencia de otros sistemas parecidos, no necesita declarar un esquema de la información que añadimos, pero para sacar mayor partido a la información tendremos que añadir los llamados **mapping**.

- Motor de búsqueda OpenSource
- Escrito en Java
- Basado en Apache Lucene
- Permite Arquitectura: distribuida, escalable, en alta disponibilidad
- Capacidades de búsqueda y análisis en tiempo real mediante peticiones GET
- Útil para soluciones NoSQL (sin transacciones distribuidas)
- Permite utilizarlo sobre el ecosistema Hadoop para proyectos de BigData
- API RESTfull sobre http para consulta, indexación, administración en diferentes lenguajes
- Permite múltiples índices en un cluster, así como alias de índices
- Permite consultas sobre uno o varios índices
- Full Text Search o búsqueda por texto completo
- Indexa todos los campos de los documentos JSON (sin esquema rígido)
- Búsquedas mediante ElasticSearch Query DSL (Domain Specific Language): multilenguaje, geolocalización, contextual, autocompletar, etc

Su RESTful API este motor de búsqueda será ideal para realizar cualquier integración con otras tecnologías en nuestro proyecto.

- Un cluster es un conjunto de uno o más nodos que mantienen toda la información de manera distribuida e indexada.
- Un nodo es un server que forma parte de un cluster, almacena tu información y ayuda con las tareas de indexación y búsqueda del cluster.
- Un Index es una colección de documentos que tienen características similares. 

- El sharding nos permite dividir estos índices en distintas “piezas” ofreciéndonos la posibilidad de escalar horizontalmente.
- La replicación nos ofrece un mecanismo para que en caso de fallo el usuario no se vea afectado.



In [1]:
from elasticsearch_dsl import connections

connections.create_connection(hosts=['elasticsearch'], timeout=20)

<Elasticsearch([{'host': 'elasticsearch'}])>

In [2]:
print(connections.get_connection().cluster.health())

{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [6]:
from datetime import datetime
from elasticsearch_dsl import Document, Date, Integer, Keyword, Text
from elasticsearch_dsl.connections import connections

class Article(Document):
    title = Text(analyzer='snowball', fields={'raw': Keyword()})
    body = Text(analyzer='snowball')
    tags = Keyword()
    published_from = Date()
    lines = Integer()

    class Index:
        name = 'blog'
        settings = {
          "number_of_shards": 2,
        }

    def save(self, ** kwargs):
        self.lines = len(self.body.split())
        return super(Article, self).save(** kwargs)

    def is_published(self):
        return datetime.now() >= self.published_from

In [7]:
Article.init()

In [11]:
article = Article(title='Hello world!', tags=['test'])
article.body = ''' looong text '''
article.published_from = datetime.now()
article.save()

True

In [13]:
articles = Article.search()
for a in articles:
    print(article)

Article(index='blog', id='dTkqiGkBU4x9z_4Vz8eX')


In [20]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

s = Search().query("match", title="Hello")

In [25]:
print(s.to_dict())
response = s.execute()

{'query': {'match': {'title': 'Hello'}}}


In [22]:
for hit in s:
    print(hit.title)

Hello world!
